In [1]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# set the current directory
%cd "/content/drive/MyDrive/RAG (1)"

/content/drive/MyDrive/RAG (1)


In [3]:
# Install required dependencies from requirements.txt
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.

In [2]:
# Import necessary libraries
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [3]:
os.environ['GROQ_API_KEY'] = "------------------------------------------"
os.environ['PINECONE_API_KEY'] = "---------------------------------------------"

In [4]:
# Initialize the LLM
llm = ChatGroq(
    model = "qwen-2.5-32b",
    temperature=0
    )

In [5]:
# Get a simple response from the LLM
response = llm.invoke("Tell me a joke").content
print(response)

Why don't scientists trust atoms?

Because they make up everything.


In [6]:
# Function to load documents from a folder
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a specified folder
folder_path = "/content/file"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")

Loaded 113 documents from the folder.


In [7]:
# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 205 chunks.


In [12]:
# If SentenceTransformerEmbeddings throws an error run below terms and restart session
#!pip uninstall transformers
#!pip install transformers

Found existing installation: transformers 4.50.1
Uninstalling transformers-4.50.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.50.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.50.1
  Using cached transformers-4.50.1-py3-none-any.whl.metadata (39 kB)
Using cached transformers-4.50.1-py3-none-any.whl (10.2 MB)


In [8]:
# Create embeddings for the document chunks
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
#print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding

<ipython-input-8-932c3b0b728a>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Initialize Pinecone vector store
index_name = "-------------------------" # check in pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=splits,
    embedding=embedding_function,
    index_name=index_name
)

In [10]:
# Create a retriever to search the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("what is abcde method")
print(retriever_results)

[Document(id='46aa8c3d-32d9-4013-ba9b-ea9682b21b33', metadata={'author': 'Registered', 'company': 'User', 'creationdate': '2005-04-22T10:45:52-07:00', 'creator': 'Acrobat PDFMaker 7.0 for Word', 'moddate': '2005-04-22T10:47:43-07:00', 'page': 38.0, 'page_label': '39', 'producer': 'Acrobat Distiller 7.0 (Windows)', 'source': '/content/file/Eat That Frog.pdf', 'sourcemodified': 'D:20050422174457', 'title': 'Eat That Frog', 'total_pages': 113.0}, page_content='Chapter 5 - Practice the ABCDE Method Continually \n \n“The first law of success is concentration – to bend all the energies to one \npoint, and to go directly to that point, looking neither to the right or to the \nleft.” (William Mathews) \n \nThe more thought you invest in planning and setting priorities before \nyou begin, the more important things you will do and the faster you \nwill get them done once you get started. \n \nThe more important and valuable the task is to you, the more you \nwill be motivated to overcome procras

In [11]:
# Create a chain to contextualize questions based on chat history
contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history,
formulate a standalone question that captures all necessary context from the chat history.
If the question is ambiguous, ask clarifying questions to resolve ambiguity.
Do NOT answer the question; just reformulate it if needed.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "what is abcde method?", "chat_history": []}))

Could you please specify which "abcde method" you are referring to? There are different "abcde methods" across various fields such as psychology, education, and even in programming. Providing more context would help in giving you the most relevant information.


In [12]:
# Create a history-aware retriever that uses the contextualized question
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Create a QA chain to answer questions based on retrieved context
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [13]:
# Initialize chat history
chat_history = []

# First question-answer pair
question1 = "what is a frog"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

# Second question-answer pair, leveraging the chat history
question2 = "what do you mean by eating it"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")

Human: what is a frog
AI: In the context provided from Brian Tracy's book "Eat That Frog," a "frog" is a metaphor for a task or responsibility that you find unpleasant or difficult but need to accomplish. The phrase "eat that frog" suggests tackling your most challenging or undesirable task first thing, so that the rest of your day or workload feels easier in comparison. The idea is to prioritize and conquer your most important or daunting tasks to enhance productivity and reduce procrastination.

Human: what do you mean by eating it
AI: In the context of Brian Tracy's book "Eat That Frog," "eating a frog" means tackling and completing your most important or unpleasant task first. The idea is to prioritize and get the most challenging or undesirable task out of the way early, so the rest of your day or workload feels easier and more manageable. By doing so, you enhance your productivity and reduce the stress and procrastination often associated with avoiding difficult tasks.


Run streamlit on colab

In [14]:
!pip install streamlit


In [15]:
print("Tunnel Password:")
!wget -q -O - ipv4.icanhazip.com

Tunnel Password:
35.225.244.222


In [16]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.225.244.222:8501

your url is: https://tall-frogs-speak.loca.lt
/content/drive/MyDrive/RAG (1)/app1.py:98: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
2025-03-26 11:04:49.350351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742987089.394209    4859 cuda_dnn.cc:8310]